# Single Train Gradient Update

## Importing Libraries

In [1]:
from typing import Dict
import threading

import gym
import numpy as np
import torch as th

from stable_baselines3 import A2C as ALGO
from stable_baselines3.common.evaluation import evaluate_policy

## Init. ENV and Model

In [2]:
env = gym.make('CartPole-v1')
model = ALGO(
    "MlpPolicy",
    env
)

## Functions to Evaluate Model and Train Model within Thread

In [3]:
def evaluate(model, env, message = '', verbose = 0):
    fitnesses = []
    iterations = 10
    for i in range(iterations):
        fitness, _ = evaluate_policy(model, env)
        if verbose == 1:
            print(i, fitness, end=" ")
        fitnesses.append(fitness)

    mean_fitness = np.mean(sorted(fitnesses))
    print(f'Type {message} Mean reward: {mean_fitness}')

In [4]:
def train(model, timesteps):
    # print('Starting Training')
    model.learn(total_timesteps=timesteps)
    # print('Completed Training')

## Initial Evaluation

In [5]:
evaluate(model, env)

/home/arnab/.miniconda3/envs/pydrl/lib/python3.7/site-packages/stable_baselines3/common/evaluation.py:69: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  UserWarning,


Type  Mean reward: 113.29


## Train for 1K Steps and Evaluate

In [6]:
# Train MT Model 1
t1 = threading.Thread(target=train, args=(model, 10_00))

# starting thread
t1.start()

# wait until thread is completely executed
t1.join()


# model_trained.learn(total_timesteps=10_00)
evaluate(model, env)

Type  Mean reward: 9.38


## Apply Gradient and Evaluate

In [7]:
for i in range(100):
    print('Train Iter: ', i, end="| ")

    # Train MT Model 1
    t1 = threading.Thread(target=train, args=(model, 50))

    # starting thread
    t1.start()
    
    # wait until thread is completely executed
    t1.join()
    
    evaluate(model, env, 'Updated Model', verbose=0)

Train Iter:  0| Type Updated Model Mean reward: 9.170000000000002
Train Iter:  1| Type Updated Model Mean reward: 9.33
Train Iter:  2| Type Updated Model Mean reward: 9.620000000000001
Train Iter:  3| Type Updated Model Mean reward: 9.350000000000001
Train Iter:  4| Type Updated Model Mean reward: 9.46
Train Iter:  5| Type Updated Model Mean reward: 9.819999999999999
Train Iter:  6| Type Updated Model Mean reward: 9.31
Train Iter:  7| Type Updated Model Mean reward: 10.09
Train Iter:  8| Type Updated Model Mean reward: 9.870000000000001
Train Iter:  9| Type Updated Model Mean reward: 9.82
Train Iter:  10| Type Updated Model Mean reward: 9.25
Train Iter:  11| Type Updated Model Mean reward: 9.37
Train Iter:  12| Type Updated Model Mean reward: 9.399999999999999
Train Iter:  13| Type Updated Model Mean reward: 9.5
Train Iter:  14| Type Updated Model Mean reward: 9.489999999999998
Train Iter:  15| Type Updated Model Mean reward: 9.370000000000001
Train Iter:  16| Type Updated Model Mean r

In [8]:
model.get_parameters()

{'policy': OrderedDict([('mlp_extractor.policy_net.0.weight',
               tensor([[-1.8331e-01, -1.6507e-01, -4.6408e-01,  8.8575e-02],
                       [ 1.7446e-01,  2.9289e-01,  2.1712e-01, -1.7124e-01],
                       [ 1.5389e-01,  1.4528e-01,  2.4494e-01, -4.4658e-02],
                       [-1.9655e-01, -9.2355e-02,  2.5860e-01,  3.4209e-01],
                       [ 5.3109e-02,  5.8897e-02,  1.2367e-01,  2.2904e-01],
                       [-1.2061e-01, -2.9573e-02, -1.2578e-01, -1.0536e-01],
                       [-7.9987e-02,  2.6514e-01,  5.0527e-02,  2.0904e-01],
                       [ 1.0737e-01, -3.9490e-02,  1.0363e-02, -2.4301e-01],
                       [-2.0188e-01, -1.9806e-01, -4.7069e-03, -1.1911e-01],
                       [ 2.7154e-03,  1.7215e-01,  1.7344e-01,  3.0100e-02],
                       [ 1.8351e-01, -6.1347e-02,  5.2896e-02, -4.4096e-01],
                       [ 1.0752e-01, -2.6380e-01,  2.3568e-01,  1.1084e-01],
              

In [9]:
model.save('a2c_lunar_singleproc')

In [10]:
# Exporting Params as JSON
## Function to Convert Params Dict to Flattened List
def flatten_list(params):
    """
    :param params: (dict)
    :return: (np.ndarray)
    """
    params_ = {}
    for key in params.keys():
        params_[key] = params[key].tolist()
    return params_
## Write Parameters to JSON File
import json

all_params = model.get_parameters()
pol_params = flatten_list(all_params['policy'])

all_params['policy'] = pol_params

with open('a2c_lunar_singleproc.json', 'w') as f:
    json.dump(all_params, f, indent='\t')

TypeError: Object of type Tensor is not JSON serializable

In [ ]:
model_loaded = ALGO(
    "MlpPolicy",
    env
)

evaluate(model_loaded,env, verbose=1)

new_params = all_params
loaded_pol_params = new_params['policy']
for key in loaded_pol_params.keys():
    loaded_pol_params[key] = th.tensor(loaded_pol_params[key])

new_params['policy'] = loaded_pol_params

model_loaded.set_parameters(new_params)

In [ ]:
env.reset()
evaluate(model_loaded,env, verbose=1)